In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load the trained model and scaler
model = joblib.load("random_forest_optimized.pkl")  # Ensure the model file exists
scaler = joblib.load("scaler.pkl")

# Initialize Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return "Diabetes Prediction API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    """
    Endpoint to predict diabetes based on input features.
    Expected JSON input format:
    {
        "features": [Age, BMI, HBA1C, Fasting_Blood_Sugar, Cholesterol_Level]
    }
    """
    try:
        # Get JSON data
        data = request.json

        # Validate input
        if "features" not in data:
            return jsonify({"error": "Missing 'features' key in request JSON"}), 400

        # Convert input to numpy array and reshape for prediction
        input_features = np.array(data["features"]).reshape(1, -1)

        # Scale input data
        input_scaled = scaler.transform(input_features)

        # Predict using the loaded model
        prediction = model.predict(input_scaled)[0]

        # Return the result as JSON
        result = {"prediction": "Diabetic" if prediction == 1 else "Non-Diabetic"}
        return jsonify(result)

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(debug=True)
